## 初めに

Kaggle初心者の練習なので、何卒お手柔らかにお願いいたします。

## Setup

In [1]:
%%capture
!pip install --user pycaret -full
!pip install numba==0.53

In [2]:
import pycaret.classification as cf

In [3]:
import numpy as np
import pandas as pd
import warnings

In [4]:
def log_mem(txt):
    import psutil
    mem = psutil.virtual_memory()
    print(txt, mem.percent, '%', mem.used//1024//1024, 'MB')

In [5]:
warnings.simplefilter('ignore')
%matplotlib inline

In [6]:
# データセット読み込み
train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv", index_col='id')
test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv", index_col='id')
sub = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")
display(train.head())
display(test.head())
display(sub.head())

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,1,3,3,3,1,6,1,0,7,4,0.298218,-0.919717,3.058541,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,4,0,2,3,0,1,0,4,6,0,-3.147667,-1.075434,2.179050,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,2,6,6,4,3,1,2,2,1,4,2.820733,-3.485342,-0.784235,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,1,0,1,6,4,2,3,3,0,3,1.081084,-2.100177,-2.343819,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,0,4,3,0,6,0,3,3,1,0,-0.126179,0.605033,1.133665,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1


,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
900000,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,0,0,0,5,1,1,2,2,0,1,-1.128371,2.749347,-1.006400,-1.193879,-2.435736,-2.427430,-1.966887,5.734205,BAAABADLAC,99.478419,0,0
900001,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,4,0,2,1,3,0,0,0,2,0,-4.424098,1.080762,2.382405,0.149442,1.883322,-2.848714,-0.725155,3.194219,AFABBAEGCB,-65.993825,1,0
900002,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,4,1,1,3,2,2,4,1,0,3,-1.523864,-1.406712,-7.026098,1.312277,-5.157192,1.714005,0.585032,0.066898,BBACABBKEE,-87.405622,0,1
900003,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,4,2,1,5,0,3,3,1,4,2,-1.404597,3.011085,-0.594532,-3.939475,1.754570,-2.364007,-1.003320,3.893099,AEBEAACQCC,-281.293460,0,0
900004,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,2,0,0,3,0,1,2,0,2,2,-1.968516,0.100594,0.084906,-0.985736,-0.130467,-3.557893,1.210687,1.861884,AEBBBBDABF,25.629415,0,2


,id,target
0,900000,0.5
1,900001,0.5
2,900002,0.5
3,900003,0.5
4,900004,0.5


In [7]:
# f_27が文字列、それ以外は数値
train.dtypes

f_00      float64
f_01      float64
f_02      float64
f_03      float64
f_04      float64
f_05      float64
f_06      float64
f_07        int64
f_08        int64
f_09        int64
f_10        int64
f_11        int64
f_12        int64
f_13        int64
f_14        int64
f_15        int64
f_16        int64
f_17        int64
f_18        int64
f_19      float64
f_20      float64
f_21      float64
f_22      float64
f_23      float64
f_24      float64
f_25      float64
f_26      float64
f_27       object
f_28      float64
f_29        int64
f_30        int64
target      int64
dtype: object

In [8]:
# カーディナリィが高いカテゴリ変数、f_27が混ざっており、このままだとメモリ不足する。
# また、整数値のうちカテゴリ変数は29, 30のみであり、7～18は数値として扱うべきである。
train.nunique()

f_00      900000
f_01      900000
f_02      900000
f_03      900000
f_04      900000
f_05      900000
f_06      900000
f_07          16
f_08          16
f_09          15
f_10          15
f_11          14
f_12          16
f_13          13
f_14          14
f_15          15
f_16          16
f_17          14
f_18          14
f_19      900000
f_20      900000
f_21      900000
f_22      900000
f_23      900000
f_24      900000
f_25      900000
f_26      900000
f_27      741354
f_28      900000
f_29           2
f_30           3
target         2
dtype: int64

In [9]:
# f_27を、文字の出現回数等の数値として扱う
def c2i(c):
    return ord(c) - ord('A')

def calc_str_std(str):
    import string
    ret = 0
    for A2Z in string.ascii_uppercase:
        val = str.count(A2Z)
        ret += val * val
    return ret**0.5

def calc_str_max(str):
    import string
    ret = 0
    for A2Z in string.ascii_uppercase:
        val = str.count(A2Z)
        ret = max(ret, val)
    return ret

def calc_unique_str(str):
    return len(set(str))

def calc_freq_str(str):
    from collections import Counter
    cnt_of = Counter(str)
    ret = max(cnt_of.values())
    return ret

def calc_linear_regression(str):
    arr = [c2i(c) for c in str]
    y = np.array(arr)
    x = np.arange(len(y))
    result = np.polyfit(x, y, 1)
    return tuple(result.tolist())

def separete_str(str):
    ret = []
    for i in range(len(str)):
        ret.append(float(c2i(str[i])))
    return tuple(ret)

def fe(data):
    import string
    for c in string.ascii_uppercase:
        data['f_27_' + c] = data['f_27'].apply(lambda x: x.count(c)).astype(np.float16)
    data['f_27_std'] = data['f_27'].apply(calc_str_std)
    data['f_27_max'] = data['f_27'].apply(calc_str_max)
    data['f_27_unique'] = data['f_27'].apply(calc_unique_str).astype(np.float16)
    data['f_27_freq'] = data['f_27'].apply(calc_freq_str).astype(np.float16)
    # result_type='expand'で、複数列同時に編集できちゃうらしい。すごいね
    data[['f_27_linear_regression_a', 'f_27_linear_regression_b']] = data.apply(lambda x: calc_linear_regression(x['f_27']), axis=1, result_type='expand')
    column_list = []
    for i in range(10):
        column = 'f_27_pos_' + str(i)
        column_list.append(column)
    train[column_list] = train.apply(lambda x: separete_str(x['f_27']), axis=1, result_type = "expand")
    test[column_list] = test.apply(lambda x: separete_str(x['f_27']), axis=1, result_type = "expand")
    return data

train = fe(train)
test = fe(test)

In [10]:
# f_27は加工して既に突っ込んだので、不要
train.drop(['f_27'], axis=1, inplace=True)
test.drop(['f_27'], axis=1, inplace=True)

In [11]:
# int型の列は、カテゴリ変数ではなく数値として扱ったほうがよさそう
numeric_features = list(train.select_dtypes(include=['float64', 'int64']).columns)
numeric_features.remove('target')

In [12]:
train.shape

(900000, 73)

In [13]:
log_mem('bef_setup')

bef_setup 12.0 % 1647 MB


PyCaret

英語だけど・・・使い方

https://pycaret.gitbook.io/docs/

https://pycaret.readthedocs.io/en/latest/api/classification.html

In [14]:
# PyCaretで分析開始
# 初期化
cf1 = cf.setup(train,
                target = 'target',
                use_gpu = True,
                session_id = 123,
                numeric_features = numeric_features,  # 数値として与える整数値
                # 1. 欠損値の処理
                # imputation_type='iterative',  # 代入の方法、simple, iterative 詳細は不明
                # numeric_imputation = 'mean',  # 数値データの補完方法、mean, median
                # categorical_imputation = 'constant',  # カテゴリデータの補完方法、constant, mode
                # 2. 順序データのエンコード
                # ordinal_features = { 'column_name' : ['low', 'medium', 'high'] },
                # 3. 特徴量の正規化
                # normalize=True,  # 正規化を行うか否か
                # zscore->普通の正規化、minmax->[0, 1]の範囲でスケーリング、maxabs->平行移動させず、絶対値を1にする、robust->四分位を使い外れ値はそのまま残す
                # normalize_method = 'robust',  # zscore, minmax, maxabs, robust
                # 4. カテゴリ変数のレア値の統合
                # combine_rare_levels = True,  # 統合を行う
                # rare_level_threshold = 0.1  # 統合を行う閾値
                # 5. 数値データのビン化
                # bin_numeric_features = ['column1', 'column2', 'column3'],  # 数値データをカテゴリ変数化させる
                # 6. 外れ値の除去
                # remove_outliers = True
                # outliers_threshold = 0.05  # 外れ値を除去する閾値
                # 7. マルチコの除去
                # remove_multicollinearity = True,  # 多変量性を除外する
                # multicollinearity_threshold = 0.9
                # remove_perfect_collinearity = True,  # 完全相関を除外する
                # 8. クラスタリング結果の特徴量化
                # create_clusters = True,  # 各特徴量をクラスタリングし、それを新たな特徴量として追加する
                # cluster_iter = 20,
                # 9. 統計的に有意ではない分散を持つ特徴量を削除
                # ignore_low_variance = True,  # 統計的に有意ではない分散を持つ特徴量を削除
                # 10. 相互作用特徴量の生成
                # 特徴量が多いとフリーズするので注意
                # polynomial_features = True, # 多項式特徴量を作成する
                # polynomial_degree = 2,  # 多項式の次数
                # trigonometry_features = True,  # 三角関数特徴量を作成する
                # polynomial_threshold = 0.1,  # 多項式特徴量を作成する閾値　低いほど少ない
                # feature_interaction = False,  # パラメータに、変数間の積を追加する
                # feature_ratio = False,  # パラメータに、変数間の比率を追加する
                # interaction_threshold = 0.0001,  # 積や比率を追加する閾値 default:0.01
                # 11. グループ特徴量の生成
                # TODO
                # 12. 特徴量選択
                feature_selection= True, # 特徴量選択を行う。パラメータをかさましした時はTrue推奨
                # feature_selection_threshold = 0.7,  # 特徴量選択後に使われるパラメータ数、大きいほどたくさん残る default 0.8
                # 13. 高カーディナリィ特徴量の削減
                # 種類数が多すぎるカテゴリ変数をきれいにする
                # high_cardinality_features = ['column1', 'column2', 'column3'],
                # high_cardinality_method = 'frequency',  # frequency, clustering
                # 14. 特徴量スケーリング
                # 正規分布を仮定しして変換する。normalizeと何が違うの？
                # transformation = False,
                # transformation_method = 'yeo-johnson',  # yeo-johnson, quantile
                # 15. 目的変数のスケーリング
                # transcorm_target = True,
                # transform_target_method = 'box-cox',  # box-cox, yeo-johnson
                # 16. 特徴量の次元削減
                # pca = True,  # PCAを行い、パラメータの次元圧縮を行う
                # pca_method = 'kernel'  # 'linear', 'kernel', 'incremental'
                # pca_components = 0.99  # 残す特徴量の割合
                # 17. その他の設定
                fold = 3,
                silent = True  # 対話的な設定を無効化
                )

,Description,Value
0,session_id,123
1,Target,target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(900000, 73)"
5,Missing Values,False
6,Numeric Features,72
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [15]:
log_mem('aft_setup')

aft_setup 23.0 % 3410 MB


In [16]:
# SetUpで補正後のデータ
cf.get_config('X').head()

,f_11,f_27_D,f_27_pos_5,f_08,f_27_V,f_27_unique,f_27_pos_8,f_27_N,f_27_Y,f_27_Z,...,f_27_P,f_13,f_14,f_27_F,f_27_C,f_27_pos_6,f_27_std,f_15,f_21,f_03
id,,,,,,,,,,,,,,,,,,,,,
0,3.0,2.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,1.0,6.0,0.0,0.0,3.0,6.000000,1.0,3.058541,0.567405
1,2.0,1.0,0.0,3.0,0.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,4.0,3.0,5.291502,0.0,2.179050,-0.545661
2,6.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,0.0,1.0,2.0,5.477226,2.0,-0.784235,0.810492
3,1.0,1.0,0.0,2.0,0.0,4.0,2.0,0.0,0.0,0.0,...,0.0,4.0,2.0,0.0,2.0,2.0,5.830952,3.0,-2.343819,2.086822
4,3.0,1.0,1.0,3.0,0.0,6.0,5.0,0.0,0.0,0.0,...,0.0,6.0,0.0,1.0,2.0,2.0,4.898980,3.0,1.133665,-1.101343


In [17]:
cf.get_config('X').dtypes

f_11          float32
f_27_D        float16
f_27_pos_5    float32
f_08          float32
f_27_V        float16
               ...   
f_27_pos_6    float32
f_27_std      float32
f_15          float32
f_21          float32
f_03          float32
Length: 71, dtype: object

PyCaretで、機械学習を行う

使うモデルは、xgboost, catboost, lightgbm, rf
(事前にcf.compare_modelsで評価が良かったことを確認している)

4モデルを作成した後、stack_model(スタッキング)->tune_model(ハイパーパラメータ調整)->calibrate_model(較正)->finalize_model(全データを対象に学習)->predict_model(testデータを対象に予測)する。

10行以下で出来て素敵

In [18]:
# model_list = cf.compare_models(n_select = 4)
model_list = [cf.create_model(model_name) for model_name in ['xgboost', 'catboost', 'lightgbm', 'rf']]
log_mem('aft_model_list')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8365,0.9141,0.8033,0.8523,0.8271,0.6723,0.6734
1,0.8353,0.9130,0.8028,0.8503,0.8259,0.6698,0.6708
2,0.8359,0.9142,0.8015,0.8525,0.8262,0.6711,0.6722
Mean,0.8359,0.9138,0.8025,0.8517,0.8264,0.6711,0.6721
Std,0.0005,0.0005,0.0008,0.0010,0.0005,0.0010,0.0010


aft_model_list 33.1 % 4565 MB


In [19]:
stacker = cf.stack_models(model_list, meta_model=model_list[0])
log_mem('aft_stack')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9309,0.9827,0.9270,0.9307,0.9289,0.8617,0.8617
1,0.9292,0.9820,0.9265,0.9280,0.9272,0.8583,0.8583
2,0.9305,0.9826,0.9258,0.9311,0.9284,0.8609,0.8609
Mean,0.9302,0.9824,0.9264,0.9299,0.9282,0.8603,0.8603
Std,0.0007,0.0003,0.0005,0.0014,0.0007,0.0014,0.0015


aft_stack 39.7 % 5603 MB


In [20]:
# tuned_model = cf.tune_model(stacker)
tuned_model = stacker  # あんまり差がなさそうだから一旦飛ばす

In [21]:
# calib_model = cf.calibrate_model(tuned_model)
calib_model = tuned_model  # あんまり差がなさそうだから一旦飛ばす

In [22]:
log_mem('bef_finalize')
final_model = cf.finalize_model(calib_model)
log_mem('aft_finalize')

bef_finalize 39.7 % 5594 MB
aft_finalize 39.8 % 5615 MB


In [23]:
# 本コンペで必要なのはtargetが1になる確率なので、raw_score=Trueを設定し確率を出力する。
pred =cf.predict_model(final_model, data=test, raw_score=True)
pred.head()
log_mem('aft_pred')

aft_pred 39.8 % 5620 MB


In [24]:
# 提出用csvの作成
sub['target'] = pred.reset_index()['Score_1']
sub.to_csv('submission.csv', index=False)
sub

,id,target
0,900000,0.9671
1,900001,0.9940
2,900002,0.0014
3,900003,0.0050
4,900004,0.9942
...,...,...
699995,1599995,0.9460
699996,1599996,0.9988
699997,1599997,0.3190
699998,1599998,0.0735
